In [ ]:
!pip install fastapi
!pip install colabcode

In [2]:
path = "/content/drive/MyDrive/font-recognition-model/"

In [3]:
import sys
sys.path.append(path)

In [4]:
import pickle
import torch
import cv2
import numpy as np
from fastapi import FastAPI, UploadFile, File
from colabcode import ColabCode
from pydantic import BaseModel
from pydantic import Field
from font_recognition import Preprocessing, PyTorchClassifier

In [5]:
!ngrok config add-authtoken 2gSxMRDMrnaQ4xVXeGvAbLFnehh_2tWjPGWBhHFVRkYJzBgwm

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
app = FastAPI()

# Labels
labels = ['Scheherazade New', 'Marhey', 'Lemonada', 'IBM Plex Sans Arabic']

# Load the preprocessor pipeline
with open(f"{path}preprocess_pipe.pkl", "rb") as f:
    loaded_pipeline = pickle.load(f)

# Initialize the preprocessor
preprocessor = Preprocessing(loaded_pipeline)

# Initialize the pytorch model
pytorch_classifier = PyTorchClassifier(2981, 512, 256, 4 , learning_rate=0.00025, epoch=50)
pytorch_classifier.load_state_dict(torch.load(f"{path}best_model.pth"))
pytorch_classifier.eval()


@app.get("/")
def read_items():
    return "Welcome to the Font Recognition API"

# Input Schema is an image file
class Image(BaseModel):
    file: UploadFile = File(..., title="Image file to be uploaded")

# Output Schema
class Prediction(BaseModel):
    pred : str = Field(..., title="Font Prediction", example="Lemonada")

@app.post("/predict_font", response_model=Prediction)
async def predict_font(file: UploadFile = File(...)):
    try:
        # Read the image file using cv2 and convert to grayscale
        contents = await file.read()
        image = cv2.imdecode(np.frombuffer(contents, np.uint8), cv2.IMREAD_GRAYSCALE)
        # Preprocess the image
        image = preprocessor.preprocess_test_data(image)
        # Predict the font
        pred = pytorch_classifier.predict(image)
        print(pred)
        return {"pred": labels[pred[0]]}
    except Exception as e:
        return {"error": str(e)}

In [7]:
server = ColabCode(port=8000, code=False)
server.run_app(app=app)

INFO:     Started server process [1221]
INFO:uvicorn.error:Started server process [1221]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://37b5-35-203-166-189.ngrok-free.app" -> "http://localhost:8000"
INFO:     102.186.233.105:0 - "GET / HTTP/1.1" 200 OK
INFO:     102.186.233.105:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     102.186.233.105:0 - "GET /openapi.json HTTP/1.1" 200 OK


100%|██████████| 1/1 [00:00<00:00,  3.12it/s]


[0]
INFO:     102.186.233.105:0 - "POST /predict_font HTTP/1.1" 200 OK


100%|██████████| 1/1 [00:00<00:00,  6.36it/s]


...

100%|██████████| 1/1 [00:00<00:00,  3.41it/s]


[3]
INFO:     102.186.233.105:0 - "POST /predict_font HTTP/1.1" 200 OK
INFO:     102.186.233.105:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     102.186.233.105:0 - "GET /openapi.json HTTP/1.1" 200 OK


100%|██████████| 1/1 [00:00<00:00,  3.59it/s]


[3]
INFO:     102.186.233.105:0 - "POST /predict_font HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [1221]
INFO:uvicorn.error:Finished server process [1221]
